# **<span style = "color:red">Inbound Reports</span>**

<div class = "alert alert-block alert-info" style = "margin-top:2-px">
    
<span style= "color:blue">1. Imports</span>

<span style = "color:blue">2. Data wrangling</span>

<span style = "color:blue">3. Plotly Dashboard</span>
    
<span style = "color:blue">4. Ratio</span>
    
<span style = "color:blue">5. Putaway ASN and HU</span>
    
<span style = "color:blue">6. Checking ASN and HU</span>


## <span style = 'color:orange'>1.Imports</span>

In [ ]:
import pandas as pd 
from datetime import timedelta,datetime
import numpy as np
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from fuzzywuzzy import fuzz




Date = "2021-10-26"
min_date = "2021-10-26"
max_date = "2021-11-27"

## <span style = 'color:orange'>2.Data Wrangling</span>

In [ ]:
df = pd.read_excel("1.ASNDeliveries.xlsx")
Deliveries = df[['ASNNo','CreatedDate','StatusDesc','TruckArrivalDate','GRNReady']]
Deliveries['CreatedDate'] = pd.to_datetime(Deliveries['CreatedDate'])
Deliveries = Deliveries[Deliveries['GRNReady']=="No"]
Deliveries.reset_index(inplace = True)
Deliveries.drop(columns = ['index'], inplace=True)

df1 = pd.read_excel("2.ASNLines.xlsx")
All_Deliveries = df1[['ASNNo','StatusDesc','NumberOfLines','NumberOfReceivedLines']]
Match = []
for i in Deliveries['ASNNo']:
    for t in All_Deliveries['ASNNo']:
        if i == t:
            Match.append(t)
            
Match_df =pd.DataFrame(Match, columns = ['ASNNo'])
Missing_Del = Deliveries[~Deliveries['ASNNo'].isin(Match_df['ASNNo'])]

f = []
for i in Match_df['ASNNo']:
    a = All_Deliveries[All_Deliveries['ASNNo']==i]
    f.append(a)
    
NOT_GRNED = pd.concat(f)
NOT_GRNED.reset_index(drop=True, inplace = True)

Open_route = All_Deliveries[All_Deliveries['StatusDesc'] == "New"]
To_be_Delivered = Open_route[~Open_route['ASNNo'].isin(NOT_GRNED['ASNNo'])]
To_be_Delivered1 = To_be_Delivered[~To_be_Delivered['ASNNo'].isin(df1[df1['ASNTypeDesc']=="IBT Stock"]['ASNNo'])]
Claims = To_be_Delivered[~To_be_Delivered['ASNNo'].isin(To_be_Delivered1['ASNNo'])]


NOT_GRNED = pd.merge(NOT_GRNED, Deliveries, on='ASNNo', how='outer')
NOT_GRNED['StatusCode'] = NOT_GRNED['StatusDesc_y'].replace(['New','Closed','Work In Progress'],['N','C','WIP'])
NOT_GRNED['ASNNo_Status'] = NOT_GRNED['ASNNo']+"   ("+NOT_GRNED['StatusCode']+")"
NOT_GRNED['Ageing'] = datetime.today() - NOT_GRNED['CreatedDate']
NOT_GRNED['Ageing'] = NOT_GRNED['Ageing'].dt.days.astype('int16')




ASN_Rec = pd.read_excel("4.ASN_Rec.xlsx")
ASN_Rec['Date Received'] = pd.to_datetime(ASN_Rec['Date Received'])
ASN_Rec['Duplicates'] = ASN_Rec['ASNNo'] + ASN_Rec['ProductCode']
ASN_Rec.drop_duplicates(subset = 'Duplicates', keep = 'last', inplace  = True)
ASN_Rec['Date Received'] = ASN_Rec['Date Received'].dt.strftime('%Y-%m-%d')


ASN_Put = pd.read_excel("5.ASN_Put.xlsx")
ASN_Put['Put-Away Date'] = pd.to_datetime(ASN_Put['Put-Away Date'])
ASN_Put['Duplicates'] = ASN_Put['ASNNo'] + ASN_Put['ProductCode']
ASN_Put.drop_duplicates(subset = 'Duplicates', keep = 'last', inplace = True)
ASN_Put['Put-Away Date'] = ASN_Put['Put-Away Date'].dt.strftime('%Y-%m-%d')

HU_Rec = pd.read_excel("6.HU_Rec.xlsx")
HU_Put = pd.read_excel("7.HU_Put.xlsx")


HU_Put['PutawayDate'] = pd.to_datetime(HU_Put['PutawayDate'])
HU_Put['PutawayDate'] = HU_Put['PutawayDate'].dt.strftime('%Y-%m-%d')
HU_Rec['ReceivingDate'] = pd.to_datetime(HU_Rec['ReceivingDate'])
HU_Rec['ReceivingDate'] = HU_Rec['ReceivingDate'].dt.strftime('%Y-%m-%d')


Yellow_Fin = pd.read_excel("8.Yellow_Ageing.xlsx")


local = df1[df1['ASNTypeDesc'] == 'Local']
local = local[['ASNNo','StatusDesc','NumberOfLines','NumberOfReceivedLines']]
local = local[(local['StatusDesc'] == "In Progress")|(local['StatusDesc']=="New")|(local['StatusDesc']=="Closed")]


claims = df1[df1['ASNTypeDesc'] == 'IBT Stock']
claims = claims[['ASNNo','StatusDesc','NumberOfLines','NumberOfReceivedLines']]
claims = claims[(claims['StatusDesc'] == "In Progress")|(claims['StatusDesc']=="New")|(claims['StatusDesc']=="Closed")]

POLines = pd.read_excel("9.POLines.xlsx")
POLines = POLines[POLines['GRNReady'] == 'No']
POLines = POLines[['ASNNo','CreatedDate','StatusDesc','TruckArrivalDate','GRNReady']]
POLines = pd.merge(POLines,All_Deliveries, on = 'ASNNo', how  = 'outer')
POLines = POLines[POLines['GRNReady']=="No"]
POLines["Code"] = POLines['StatusDesc_y'].replace(['In Progress','New','Closed'],['WIP','N','C'])
POLines['ASNNo_Status'] = POLines['ASNNo']+" ("+POLines['Code']+")"
POLines['CreatedDate'] = pd.to_datetime(POLines['CreatedDate'])
POLines['Ageing'] = datetime.today() - POLines['CreatedDate']
POLines['Ageing'] = POLines['Ageing'].dt.days.astype('int16')



OpenRoutes = pd.read_excel("3.OpenRoutes.xlsx")
OpenRoutes['Route Create Date'] = pd.to_datetime(OpenRoutes['Route Create Date'])
OpenRoutes['Ageing'] = datetime.today() - OpenRoutes['Route Create Date']
OpenRoutes['Ageing'] = OpenRoutes['Ageing'].dt.days.astype('int16')

## <span style = 'color:orange'>3.Plotly Dashboard</span>

In [ ]:
app = dash.Dash(external_stylesheets = [dbc.themes.JOURNAL],)


PLOTLY_LOGO = "https://images.plot.ly/logo/new-branding/plotly-logomark.png"

navbar = dbc.Navbar(id = 'narbar', children = [
    dbc.Row([
        dbc.Col(html.Img(src =PLOTLY_LOGO, height = '70px')),
        dbc.Col(dbc.NavbarBrand("Inbound Reports", style = {"color":"black","fontSize":"25px",
                                                            "fontFamily":"Times New Roman"}))
    ], align = "center", no_gutters = False)
])





def StatusDesc_Lines(NOT_GRNED):
    y = NOT_GRNED.groupby(['StatusDesc_y'])['NumberOfLines'].sum()
    x = y.index
    total_y = NOT_GRNED.groupby(['StatusDesc_y'])['NumberOfLines'].sum().sum().astype(str)


    fig = go.Figure([go.Bar(y=y,x=x, text = y, textposition = "auto", marker_color = 'orange')])
    fig.update_layout(title = "Deliveries Report (Lines Not GRNed )                    Total Lines:"+total_y,
                     yaxis_title = "No. of Lines",
                     xaxis_title = "StatusDesc")
    
    return fig


def StatusDesc_ASNNo(NOT_GRNED):

    y4 = NOT_GRNED.groupby(['StatusDesc_y'])['ASNNo'].count()
    x4 = y4.index

    fig = go.Figure([go.Bar(y=y4,x=x4, text = y4, textposition = "auto")])
    fig.update_layout(title={
        'text': "Deliveries Report (No. of ASNs not GRNed)",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
                     yaxis_title = "No. of ASNs",
                     xaxis_title = "StatusDesc")
    
    return fig

def ASNNo_Lines(NOT_GRNED):
    y1 = NOT_GRNED.groupby(['ASNNo_Status'])['NumberOfLines'].sum()
    y1_1 = NOT_GRNED.groupby(['ASNNo_Status'])['NumberOfReceivedLines'].sum()
    Total1 = NOT_GRNED.groupby(['ASNNo_Status'])['NumberOfLines'].sum().sum().astype(str)
    Total_Rec = NOT_GRNED.groupby(['ASNNo_Status'])['NumberOfReceivedLines'].sum().sum().astype(str)
    x1 = y1.index

    fig = go.Figure([go.Bar(y =y1, x=x1, text = y1, textposition = 'auto', name = "Total Lines"),
                    go.Bar(y =y1_1, x=x1, text = y1_1, textposition = 'auto', name = "Lines Received")])



    fig.update_layout(title = "Deliveries Report (ASN not GRNED)        Total Lines: "+Total1+" | Lines Received: "+Total_Rec,
                     xaxis_title="ASNs",
                     yaxis_title="No. of Lines")
    return  fig

def ASN_Lines_RecPut(ASN_Rec, ASN_Put):
    y = ASN_Rec.groupby(['Date Received'])['Date Received'].count()
    x = y.index
    y1 = ASN_Put.groupby(['Put-Away Date'])['Put-Away Date'].count()
    x1 = y1.index


    fig = go.Figure(data = [
        go.Bar(name = "Receiving", x =x, y = y, text = y, textposition = 'auto'),
        go.Bar(name = "Putaway", x =x1, y = y1, text = y1, textposition = 'auto')
    ])
    fig.update_layout(barmode='group',
                     title_text = "ASN Lines Receiving and Putaway",
                     xaxis_title = "Date",
                     yaxis_title = "No. of Lines",
                      xaxis_range=[min_date,max_date],
                     xaxis_tickformat = "%Y-%m-%d",
                     yaxis_range = [0,700])
    fig.add_shape(type = "line",
                  y1 = 652,
                  y0 =652, 
                  x0 = min_date,
                  x1 =max_date,
                 line=dict(color='red'))
    return fig


def HU_RecPut(HU_Rec, HU_Put):

    y4 = HU_Rec.groupby(['ReceivingDate'])['HUs'].sum().sort_values(ascending = False)
    x4 = y4.index

    y5 = HU_Put.groupby(["PutawayDate"])['HUs'].sum().sort_values(ascending = False)
    x5 = y5.index



    fig = go.Figure(data=[
        go.Bar(x = x4, y=y4, text = y4, textposition = 'auto', name ='Receiving', marker_color = 'indianred'),
        go.Bar(x = x5, y=y5, text = y5, textposition = "auto", name = "Putaway", marker_color = 'orange')
    ])
    fig.update_layout(barmode='group',
                     title_text = "HUs Receiving and Putaway",
                     xaxis_title = "Date",
                     yaxis_title = "No. of HUs",
                      xaxis_range=[min_date,max_date],
                     xaxis_tickformat = "%Y-%m-%d",
                     yaxis_range = [0,1300])
    fig.add_shape(type = "line",
                  y1 = 1152,
                  y0 =1152, 
                  x0 = min_date,
                  x1 =max_date,
                 line=dict(color='red'))
    
    return fig

def Yellow_Fin_Ageing(Yellow_Fin):
    y = Yellow_Fin.groupby(['Age in Days.'])['pr-shipno'].count()
    x = y.index
    fig = go.Figure([go.Bar(y =y,x=x, text = y, textposition = 'auto')])
    fig.update_layout(title = "Yellow_Fin Ageing Orders",
                         yaxis_title="No. orders",
                         xaxis_title ="Ageing (Days)",
                      yaxis_range = [0,40],
                      xaxis = dict(tickmode = 'linear',
                                   tick0 = 0,
                                   dtick = 1))
    
    fig.add_vline(x=2.5, line_width=3, line_dash="dash", line_color="red")
    return fig

def local_sup(local):
    y = local.groupby(['StatusDesc'])['NumberOfLines'].sum()
    total = y.sum().astype(str)
    x = y.index

    fig = go.Figure([go.Bar(y=y, x =x, text = y, textposition = 'auto')])
    fig.update_layout(title= "Local ASN Lines                                  Total Lines:"+total,
                     yaxis_title = "No. of Lines",
                     xaxis_title = "Status Desc.",
                     title_font_color="black")
    return fig


def IBT_Claims(claims):
    y = claims.groupby(['StatusDesc'])['NumberOfLines'].sum()
    total = y.sum().astype(str)
    x = y.index

    fig = go.Figure([go.Bar(y=y, x =x, text = y, textposition = 'auto')])
    fig.update_layout(title= "IBT claims ASN Lines                              Total Lines:"+total,
                     yaxis_title = "No. of Lines",
                     xaxis_title = "Status Desc.",
                     title_font_color="black")
    return fig


def Ageing_Delivery_Lines(NOT_GRNED):
    y =NOT_GRNED.groupby(['Ageing'])['NumberOfLines'].sum()
    TOTAL = y.sum().astype(str)
    x = y.index

    fig = go.Figure([go.Bar(y =y , x =x, text = y, textposition = 'auto')])

    fig.update_layout(title = "Ageing Deliveries",
                     xaxis_title = "Ageing (Days)",
                     yaxis_title = "No of Lines",
                    xaxis = dict(tickmode = 'linear',
                                 tick0 = 0,
                                 dtick = 1))
                     
    fig.add_vline(x=2.5, line_width=3, line_dash="dash", line_color="red")
    return fig


def POLines_d(POLines):

    y = POLines.groupby(['ASNNo_Status'])['NumberOfLines'].sum()
    x = y.index

    Total = y.sum().astype(str)

    y1 = POLines.groupby(['ASNNo_Status'])['NumberOfReceivedLines'].sum()
    Total1 = y1.sum().astype(str)
    x1 = y.index

    fig = go.Figure([go.Bar(y = y, x =x, text = y, textposition = 'auto', name = "Total Lines"),
                    go.Bar(y = y1, x =x , text = y1, textposition = 'auto', name = "Lines Received")])

    fig.update_layout(title = "PO lines not GRNed       Total Lines: "+Total+" | Lines Received: "+Total1,
                     xaxis_title = 'ASNs',
                     yaxis_title = 'No. of Lines')
    return fig

def POLines_Ageing(POLines):
    y = POLines.groupby(['Ageing'])['NumberOfLines'].sum()
    x = y.index


    fig = go.Figure([go.Bar(y = y, x=x, text = y, textposition = 'auto')])

    fig.update_layout(title = 'Ageing for PO lines',
                     xaxis_title= "Ageing (Days)",
                     yaxis_title = "No. of Lines",
                    xaxis = dict(tickmode = 'linear',
                                tick0 = 0,
                                dtick = 1))
    
    
    fig.add_vline(x=1.5, line_width=3, line_dash="dash", line_color="red")
    return  fig
def OpenRoutes_f(OpenRoutes):
    y = OpenRoutes.groupby(['Ageing'])['HeaderNo'].count()
    x = y.index


    fig = go.Figure([go.Bar(y =y , x =x, text = y, textposition = 'auto')])

    fig.update_layout(title = "Ageing Open Routes",
                     xaxis_title = "Ageing (Days)",
                     yaxis_title = "No. of Routes",
                     xaxis = dict(tickmode = 'linear',
                                 tick0 = 0,
                                 dtick = 1))
    fig.add_vline(x=1.5, line_width=3, line_dash="dash", line_color="red")
    return fig



app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'h1_1', children = [navbar], 
            style = {"textAlign":"center"}),
    
    html.H4(id = 'h4_1', children = "Last Update: "+Date, 
            style = {"textAlign":'center', 
                     'marginTop':25, 
                     'marginBottom':25}),
    
    html.H2(id = 'h2_1', children = 'ASNs', 
            style = {'textAlign':'center',
                     'marginTop':25, 
                     'marginBottom':25}),
    
    html.Div(id = 'StatusDesc_ASNNo_Div', children = [dcc.Graph(id = 'StatusDesc_ASNNo', 
                                                                figure = StatusDesc_ASNNo(NOT_GRNED))], 
             style = {"width":'50%',
                      "display":"inline-block"}),
    
    html.Div(id = 'StatusDesc_Lines_Div', children = [dcc.Graph(id = 'StatusDesc_Lines', 
                                                                figure = StatusDesc_Lines(NOT_GRNED))], 
             style = {"width":'50%',
                      "display":"inline-block"}),
    
    
    html.Div(id = 'ASNNo_Lines_Div', children = [dcc.Graph(id = 'ASNNo_Lines', 
                                                           figure =ASNNo_Lines(NOT_GRNED))], 
             style = {"width":'55%',
                      "display":"inline-block"}),
    
    html.Div(id = 'POLines_Div', children = [dcc.Graph(id = 'POLines', 
                                                       figure =POLines_d(POLines))], 
             style = {"width":'45%',
                      "display":"inline-block"}),
    
    html.H2(id = 'h2_2', children = "Local and IBT stock", 
            style = {"textAlign":"center", 
                     "marginTop":25, 
                     "marginBottom":25}),
    
    html.Div(id = 'Local_Div', children = [dcc.Graph(id = 'local_sup', 
                                                     figure = local_sup(local))], 
             style = {"width":'50%',
                      "display":"inline-block"}),
    
    html.Div(id = 'claim_div', children = [dcc.Graph(id = 'IBT_Claims', 
                                                     figure = IBT_Claims(claims))], 
             style = {"width":'50%',
                      "display":"inline-block"}),
    
    html.H2(id = 'h2_3', children = "ASN Lines and HUs Received and Putaway", 
            style = {"textAlign":"center", 
                     "marginTop":25, 
                     "marginBottom":25}),
    
    html.Div(id = 'ASN_Lines_RecPut_Div', children = [dcc.Graph(id = 'ASN_Lines_RecPut', 
                                                                figure = ASN_Lines_RecPut(ASN_Rec, ASN_Put))], 
             style = {'width':'50%','display':"inline-block"}),
    
    html.Div(id = 'HU_RecPut_Div', children = [dcc.Graph(id = "HU_RecPut", 
                                                         figure = HU_RecPut(HU_Rec, HU_Put))],
             style = {'width':'50%',
                      'display':"inline-block"}),
    
    html.H2(id = 'h2_4', children = "Ageing Reports", 
            style = {"textAlign":"center", 
                     "marginTop":25, 
                     "marginBottom":25}),
    
    html.Div(id = 'Yellow_Fin_Ageing_Div', children = [dcc.Graph(id = "Yellow_Fin", 
                                                                 figure = Yellow_Fin_Ageing(Yellow_Fin))],
             style = {'width':'50%',
                      'display':"inline-block"}),
    
    html.Div(id = 'Ageing_Delivery_Div', children = [dcc.Graph(id = "Ageing_Delivery",
                                                               figure = Ageing_Delivery_Lines(NOT_GRNED))],
             style = {'width':'50%',
                      'display':"inline-block"}),
    
    html.Div(id = 'POLines_Ageing_Div', children = [dcc.Graph(id = "POLines_Ageing", 
                                                              figure = POLines_Ageing(POLines))],
             style = {'width':'50%',
                      'display':"inline-block"}),
    
    html.Div(id = 'Open_Routes_Div', children = [dcc.Graph(id = "Open_Routes_f", 
                                                           figure = OpenRoutes_f(OpenRoutes))],
             style = {'width':'50%',
                      'display':"inline-block"})
])



if __name__ == "__main__":
    app.run_server()


## <span style = 'color:orange'>4. Ratio for ASNs to HUs</span>

In [ ]:
ASN_Rec_x = pd.pivot_table(ASN_Rec,index=['Date Received'],values=['Duplicates'],aggfunc='count')
ASN_Put_x = pd.pivot_table(ASN_Put, index=['Put-Away Date'], values = ['Duplicates'], aggfunc = 'count')
HU_Rec_x = pd.pivot_table(HU_Rec, index=['ReceivingDate'], values = ['HUs'], aggfunc = np.sum)
HU_Put_x = pd.pivot_table(HU_Put, index=['PutawayDate'], values = ['HUs'], aggfunc = 'count')


Put = [ASN_Put_x, HU_Put_x]
Put_ASN_HU_Ratio = pd.concat(Put, axis = 1)
Rec = [ASN_Rec_x, HU_Rec_x]
Rec_ASN_HU_Ratio = pd.concat(Rec, axis = 1)

Put_ASN_HU_Ratio['Ratio'] = Put_ASN_HU_Ratio['HUs']/Put_ASN_HU_Ratio['Duplicates']
Rec_ASN_HU_Ratio['Ratio'] =Rec_ASN_HU_Ratio['HUs']/Rec_ASN_HU_Ratio['Duplicates']

y1 = Rec_ASN_HU_Ratio.groupby(Rec_ASN_HU_Ratio.index)['Ratio'].max()
y2 = Put_ASN_HU_Ratio.groupby(Put_ASN_HU_Ratio.index)['Ratio'].max()

AverageRec = str(round(Rec_ASN_HU_Ratio.groupby(Rec_ASN_HU_Ratio.index)['Ratio'].max().mean(),2))
AveragePut = str(round(Put_ASN_HU_Ratio.groupby(Put_ASN_HU_Ratio.index)['Ratio'].max().mean(),2))
Avg_Rec = float(AverageRec)
Avg_Put = float(AveragePut)
x1 = Rec_ASN_HU_Ratio.index
x2 = Put_ASN_HU_Ratio.index


fig = go.Figure()
fig.add_trace(go.Scatter(x=x1, y=y1,
                    mode='lines',
                    name='Rec | Ratio=' + AverageRec))
fig.add_trace(go.Scatter(x=x2, y=y2,
                    mode='lines',
                    name='Put | Ratio='+AveragePut))
fig.update_layout(title_text = "Ratios for HUs to ASN Lines",
                 xaxis_title = "Date",
                 yaxis_title = "Ratio")
fig.add_hline(y=Avg_Rec, line_width=3, line_dash="dash", line_color="blue")
fig.add_hline(y=Avg_Put, line_width=3, line_dash="dash", line_color="indianred")


fig.show()

## <span style = 'color:orange'>5. Putaway ASN to HUs</span>

In [ ]:
y1 = Put_ASN_HU_Ratio.groupby([Put_ASN_HU_Ratio.index])['HUs'].max()
y2 = Put_ASN_HU_Ratio.groupby([Put_ASN_HU_Ratio.index])['Duplicates'].max()

x1 = y1.index
x2 = y2.index

Avg_HUs = str(round(y1.mean(),2))
Avg_Lines = str(round(y2.mean(),2))

Avg_HUs_float = float(Avg_HUs)
Avg_Lines_float = float(Avg_Lines)


fig = go.Figure()
fig.add_trace(go.Scatter(x=x1, y=y1,
                    mode='lines',
                    name='HUs | ' + Avg_HUs))
fig.add_trace(go.Scatter(x=x2, y=y2,
                    mode='lines',
                    name='Lines | '+Avg_Lines))
fig.update_layout(title_text = "Putaway HUs vs Lines")
fig.add_hline(y=Avg_HUs_float, line_width=3, line_dash="dash", line_color="blue")
fig.add_hline(y=Avg_Lines_float, line_width=3, line_dash="dash", line_color="indianred")


fig.show()

## <span style = 'color:orange'>6. Checking ASNs to HUs</span>

In [ ]:
y1 = Rec_ASN_HU_Ratio.groupby([Rec_ASN_HU_Ratio.index])['HUs'].max()
y2 = Rec_ASN_HU_Ratio.groupby([Rec_ASN_HU_Ratio.index])['Duplicates'].max()

x1 = y1.index
x2 = y2.index

Avg_HUs = str(round(y1.mean(),2))
Avg_Lines = str(round(y2.mean(),2))

Avg_HUs_float = float(Avg_HUs)
Avg_Lines_float = float(Avg_Lines)


fig = go.Figure()
fig.add_trace(go.Scatter(x=x1, y=y1,
                    mode='lines',
                    name='HUs | ' + Avg_HUs))
fig.add_trace(go.Scatter(x=x2, y=y2,
                    mode='lines',
                    name='Lines | '+Avg_Lines))
fig.update_layout(title_text = "Receiving HUs vs Lines")
fig.add_hline(y=Avg_HUs_float, line_width=3, line_dash="dash", line_color="blue")
fig.add_hline(y=Avg_Lines_float, line_width=3, line_dash="dash", line_color="indianred")


fig.show()